## SRJ - Building H using HF integrals

In [1]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged)
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.UHF(mol_pyscf)
rhf.kernel()

converged SCF energy = -1655.10155370991  <S^2> = 6.0386361  2S+1 = 5.0154306


np.float64(-1655.1015537099054)

In [6]:
one_ao = mol_pyscf.intor_symmetric('int1e_kin') + mol_pyscf.intor_symmetric('int1e_nuc')
two_ao = mol_pyscf.intor('int2e_sph')

print(rhf.mo_coeff.shape)  # Should be (num_AOs, num_MOs)
print(one_ao.shape)  
print(two_ao.shape)

#one_mo = np.einsum('pi,pq,qj->ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)
#one_mo = np.einsum('spi,pq,sqj->sij', rhf.mo_coeff, one_ao, rhf.mo_coeff) #Changed this 
one_mo = np.einsum('...pi,pq,...qj->...ij', rhf.mo_coeff, one_ao, rhf.mo_coeff)

two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

two_mo = np.swapaxes(two_mo, 1, 3)

core_constant = np.array([rhf.energy_nuc()])

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)

(2, 27, 27)
(27, 27)
(27, 27, 27, 27)


ValueError: too many values to unpack (expected 2)

In [3]:
print(rhf.mo_coeff.shape)  # Should be (num_AOs, num_MOs)
print(one_ao.shape)        # Check if > 2D


(2, 27, 27)
(27, 27)
